In [162]:
import csv

# Input and output file paths
input_file = "/Users/keshavsharma/Downloads/SchittVision/schitts_creek_dialogues_cleaned.csv"
output_file = "/Users/keshavsharma/Downloads/SchittVision/schitts_creek_dialogues_cleaned_final.csv"


with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        parts = line.strip().split(",", 1)  # Split only on the first comma
        if len(parts) == 2:
            character, dialogue = parts
            dialogue = dialogue.strip()
            # Add quotes if not already present
            if not (dialogue.startswith('"') and dialogue.endswith('"')):
                dialogue = f'"{dialogue}"'
            outfile.write(f"{character},{dialogue}\n")
        else:
            # Write malformed lines as-is or handle them
            print(f"Skipping malformed line: {line.strip()}")

print(f"Processed file saved to {output_file}")


Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping malformed line: 
Skipping mal

In [163]:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv(input_file)

# Merge fragmented dialogues
merged_data = []
current_character = None
current_dialogue = []

for _, row in df.iterrows():
    character, dialogue = row["Character"], row["Dialogue"]
    
    if character == current_character:  # Same character as previous
        current_dialogue.append(dialogue)
    else:
        if current_character:  # Save the last character's dialogue
            merged_data.append([current_character, " ".join(current_dialogue)])
        current_character = character
        current_dialogue = [dialogue]

# Add the final character's dialogue
if current_character:
    merged_data.append([current_character, " ".join(current_dialogue)])

# Convert back to DataFrame
merged_df = pd.DataFrame(merged_data, columns=["Character", "Dialogue"])

# Save to a new CSV
merged_df.to_csv("/Users/keshavsharma/Downloads/SchittVision/final_cleaned_dataset.csv", index=False)


In [164]:
from transformers import BertTokenizer
import pandas as pd

# Load your dataset
df = pd.read_csv("/Users/keshavsharma/Downloads/SchittVision/schitts_creek_dialogues_cleaned.csv")
 # Ensure it has 'Character' and 'Dialogue' columns

In [165]:
df.head(5)

,Character,Dialogue
0,unknown,"Episode transcripts for the TV show ""Schitt's ..."
1,Moira,"John, I've been stripped of every morsel of pl..."
2,Johnny,"Well, how do you think I feel, Moira?!"
3,Johnny,"Eli was family, for God's sake!"
4,Johnny,"Leave your finances to me, he said! Son of a b..."


In [166]:
import csv

def generate_input_target_pairs(file_path, context_window_size):
    input_target_pairs = []
    current_scene = []  # Holds dialogues for the current scene

    def process_scene(scene, window_size):
        """
        Generates input-target pairs for a single scene.
        """
        pairs = []
        for i in range(len(scene) - 1):
            # Create context window for inputs
            input_dialogues = " | ".join(
                [f"{char}: \"{dialogue}\"" for char, dialogue in scene[max(0, i - window_size + 1): i + 1]]
            )
            target_dialogue = f"{scene[i + 1][0]}: \"{scene[i + 1][1]}\""
            pairs.append((input_dialogues, target_dialogue))
        return pairs

    # Read the CSV file
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header row

        for row in reader:
            # Skip rows that are completely empty (used for scene change detection)
            if not row or len(row) < 2:
                # If current_scene has dialogues, process and reset
                if current_scene:
                    input_target_pairs += process_scene(current_scene, context_window_size)
                current_scene = []  # Reset the scene
                continue

            character, dialogue = row

            # Check for scene change: either "unknown" or empty line indicates a scene change
            if not dialogue.strip() or character == "unknown":
                # Process the current scene if it's not empty
                if current_scene:
                    input_target_pairs += process_scene(current_scene, context_window_size)
                current_scene = []  # Reset the scene
                continue

            # Group consecutive dialogues by the same character
            if current_scene and current_scene[-1][0] == character:
                current_scene[-1] = (character, f"{current_scene[-1][1]} {dialogue}")
            else:
                current_scene.append((character, dialogue))

        # Process the last scene if not already processed
        if current_scene:
            input_target_pairs += process_scene(current_scene, context_window_size)

    return input_target_pairs


# Define the file path
data_path = "/Users/keshavsharma/Downloads/SchittVision/schitts_creek_dialogues_cleaned.csv"

# Generate input-target pairs with context window size 3
context_window_size = 3
pairs = generate_input_target_pairs(data_path, context_window_size)

# Print the results
for i, (inp, target) in enumerate(pairs):
    print(f"Pair {i + 1}:")
    print(f"  Input: {inp}")
    print(f"  Target: {target}\n")


Pair 1:
  Input: Moira: "John, I've been stripped of every morsel of pleasure I earned in this life!"
  Target: Johnny: "Well, how do you think I feel, Moira?! Eli was family, for God's sake! Leave your finances to me, he said! Son of a btch!"

Pair 2:
  Input: Alexis: "Baby, it's crazy, people are just like, taking our stuff! I said, they're taking our stuff! Can you just step out of the club for a second, ple... Hold on, hold on! Those bags are not for you, my boyfriend bought those for me, so, theoretically, they are his!"
  Target: Revenue agent: "Please ma'am, let us do our job"

Pair 3:
  Input: Revenue agent: "Please sir, can you step aside?"
  Target: David: "No, you step aside! You step aside! I'm still trying to wrap my mind around what kind of a sick person wants to get paid to destroy another person's life! Destroy another person's life! Where are you taking that?!"

Pair 4:
  Input: Moira: "My very soul has been kidnapped, there's no ransom, no one's coming to save me!"
  

In [170]:
import csv

def generate_input_target_pairs(file_path, context_window_size):
    input_target_pairs = []
    current_scene = []  # Holds dialogues for the current scene

    def process_scene(scene, window_size):
        """
        Generates input-target pairs for a single scene.
        """
        pairs = []
        for i in range(len(scene) - 1):
            # Create context window for inputs
            input_dialogues = " | ".join(
                [f"{char}: \"{dialogue}\"" for char, dialogue in scene[max(0, i - window_size + 1): i + 1]]
            )
            target_dialogue = f"{scene[i + 1][0]}: \"{scene[i + 1][1]}\""
            pairs.append((input_dialogues, target_dialogue))
        return pairs

    # Read the CSV file
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header row

        for row in reader:
            # Skip rows that are completely empty (used for scene change detection)
            if not row or len(row) < 2:
                # If current_scene has dialogues, process and reset
                if current_scene:
                    input_target_pairs += process_scene(current_scene, context_window_size)
                current_scene = []  # Reset the scene
                continue

            character, dialogue = row

            # Check for scene change: either "unknown" or empty line indicates a scene change
            if not dialogue.strip() or character == "unknown":
                # Process the current scene if it's not empty
                if current_scene:
                    input_target_pairs += process_scene(current_scene, context_window_size)
                current_scene = []  # Reset the scene
                continue

            # Group consecutive dialogues by the same character
            if current_scene and current_scene[-1][0] == character:
                current_scene[-1] = (character, f"{current_scene[-1][1]} {dialogue}")
            else:
                current_scene.append((character, dialogue))

        # Process the last scene if not already processed
        if current_scene:
            input_target_pairs += process_scene(current_scene, context_window_size)

    return input_target_pairs


# Define the file path
data_path = "/Users/keshavsharma/Downloads/SchittVision/schitts_creek_dialogues_cleaned.csv"

# Generate input-target pairs with context window size 3
context_window_size = 3
pairs = generate_input_target_pairs(data_path, context_window_size)

# Save the input-target pairs to a CSV file
output_path = "input_target_pairs_final.csv"
with open(output_path, mode="w", newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Input", "Target"])  # Add headers
    writer.writerows(pairs)  # Write pairs to the CSV file

print(f"Input-Target pairs have been saved to {output_path}")


Input-Target pairs have been saved to input_target_pairs_final.csv


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import DistilBertTokenizer, DistilBertModel, AdamW

# Set device
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# Define Dataset class
class InputTargetDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_len):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        input_text = row["Input"]
        target_text = row["Target"]

        input_encoding = self.tokenizer(
            input_text, max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text, max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt"
        )

        return {
            "input_ids": input_encoding["input_ids"].squeeze(0),
            "attention_mask": input_encoding["attention_mask"].squeeze(0),
            "target_ids": target_encoding["input_ids"].squeeze(0),
            "target_attention_mask": target_encoding["attention_mask"].squeeze(0),
        }

# Load and split the dataset
print("Loading data...")
csv_file = "input_target_pairs_final.csv"
print("Initializing tokenizer...")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
max_len = 64

dataset = InputTargetDataset(csv_file, tokenizer, max_len)

# Split into Train (80%) and Validation (20%)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load fresh model before training
print("Loading model...")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
model.to(device)  # Move model to the appropriate device

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CosineEmbeddingLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    print(f"\nStarting epoch {epoch + 1}/{epochs}...")
    model.train()
    total_train_loss = 0
    optimizer.zero_grad()

    # Training Phase
    for i, batch in enumerate(train_dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        target_ids = batch["target_ids"].to(device)
        target_attention_mask = batch["target_attention_mask"].to(device)

        input_outputs = model(input_ids, attention_mask=attention_mask)
        input_vectors = input_outputs.last_hidden_state[:, 0, :]  # CLS token

        target_outputs = model(target_ids, attention_mask=target_attention_mask)
        target_vectors = target_outputs.last_hidden_state[:, 0, :]  # CLS token

        labels = torch.ones(input_vectors.size(0)).to(device)  # Similarity = 1
        loss = criterion(input_vectors, target_vectors, labels)

        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Training Loss: {avg_train_loss:.6f}")

    # Validation Phase
    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            target_ids = batch["target_ids"].to(device)
            target_attention_mask = batch["target_attention_mask"].to(device)

            input_outputs = model(input_ids, attention_mask=attention_mask)
            input_vectors = input_outputs.last_hidden_state[:, 0, :]

            target_outputs = model(target_ids, attention_mask=target_attention_mask)
            target_vectors = target_outputs.last_hidden_state[:, 0, :]

            labels = torch.ones(input_vectors.size(0)).to(device)
            loss = criterion(input_vectors, target_vectors, labels)

            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Epoch {epoch + 1} - Validation Loss: {avg_val_loss:.6f}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_distilbert_final")
tokenizer.save_pretrained("fine_tuned_distilbert")
torch.save(model, "distilbert_encoder_finetuned.pth")
print("Model and tokenizer saved successfully!")


Using device: mps
Loading data...
Initializing tokenizer...
Loading model...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Starting epoch 1/3...
Epoch 1 - Training Loss: 0.001739
Epoch 1 - Validation Loss: 0.000015

Starting epoch 2/3...
Epoch 2 - Training Loss: 0.000086
Epoch 2 - Validation Loss: 0.000004

Starting epoch 3/3...
Epoch 3 - Training Loss: 0.000037
Epoch 3 - Validation Loss: 0.000002
Model and tokenizer saved successfully!


In [231]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertModel

# Set device
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# Load the fine-tuned encoder model and tokenizer
model_path = "fine_tuned_distilbert_final"
tokenizer_path="fine_tuned_distilbert"
print("Loading fine-tuned encoder...")
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_path)
model = DistilBertModel.from_pretrained(model_path).to(device)
model.eval()  # Set to evaluation mode

# Load dataset
csv_file = "input_target_pairs_final.csv"
df = pd.read_csv(csv_file)
max_len = 64

# Function to generate context vectors
def get_context_vector(text):
    encoding = tokenizer(
        text, max_length=max_len, padding="max_length", truncation=True, return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        output = model(**encoding)
        context_vector = output.last_hidden_state[:, 0, :]  # Extract CLS token embedding

    return context_vector.cpu().numpy().flatten()  # Convert to 1D array

# Generate and store context vectors
context_vectors = []
for i, row in df.iterrows():
    input_text = row["Input"]
    context_vector = get_context_vector(input_text)
    context_vectors.append(context_vector)

# Convert to DataFrame and save
context_df = pd.DataFrame(context_vectors)
context_df["Input"] = df["Input"]
context_df["Target"] = df["Target"]

context_df.to_csv("context_vectors.csv", index=False)
print("Context vectors saved to 'context_vectors.csv'")


Using device: mps
Loading fine-tuned encoder...
Context vectors saved to 'context_vectors.csv'


In [233]:
context_df.head()

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,Input,Target
0,-0.372959,0.026562,0.371178,0.129672,0.207231,0.262574,-0.312700,-0.108629,-0.130898,-0.214850,...,0.076064,0.365417,0.123626,-0.025344,-0.229532,-0.053809,0.446565,0.331397,"Moira: ""John, I've been stripped of every mors...","Johnny: ""Well, how do you think I feel, Moira?..."
1,-0.371586,0.027815,0.369203,0.129900,0.207065,0.262231,-0.312337,-0.108022,-0.131492,-0.215416,...,0.077203,0.364914,0.123717,-0.025269,-0.230230,-0.054348,0.445849,0.330421,"Alexis: ""Baby, it's crazy, people are just lik...","Revenue agent: ""Please ma'am, let us do our job"""
2,-0.373138,0.028200,0.370433,0.130709,0.207789,0.262202,-0.312221,-0.108567,-0.130845,-0.214848,...,0.076752,0.365373,0.122891,-0.024614,-0.228327,-0.053787,0.446290,0.330560,"Revenue agent: ""Please sir, can you step aside?""","David: ""No, you step aside! You step aside! I'..."
3,-0.373176,0.026156,0.370669,0.130419,0.207036,0.262256,-0.312622,-0.109032,-0.130243,-0.215432,...,0.076250,0.365278,0.123950,-0.025086,-0.229656,-0.053324,0.446114,0.332081,"Moira: ""My very soul has been kidnapped, there...","Johnny: ""We've got 15 minutes to collect our p..."
4,-0.372870,0.026715,0.370577,0.130087,0.207519,0.262195,-0.312343,-0.108887,-0.129984,-0.214898,...,0.076688,0.364951,0.124082,-0.024845,-0.230218,-0.053470,0.446409,0.331346,"Moira: ""My very soul has been kidnapped, there...","Moira: ""Did you put Kristen with Robin? They d..."


In [196]:
from transformers import AutoTokenizer

AutoTokenizer.from_pretrained("gpt2", cache_dir="./cache")



GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [234]:
import torch
import pandas as pd

# Load your CSV
context_df = pd.read_csv("context_vectors.csv")  # Adjust filename

# Convert first 768 columns to float32 (ensure numeric type)
context_vector_columns = context_df.iloc[:, :768].astype("float32")

# Convert each row of 768 columns into a single tensor
context_df["context_vector"] = context_vector_columns.apply(
    lambda row: torch.tensor(row.values, dtype=torch.float32), axis=1
)

# Drop the first 768 columns since they're now merged into "context_vector"
context_df = context_df.drop(columns=context_df.columns[:768])

# Convert tensors back to lists before saving
context_df["context_vector"] = context_df["context_vector"].apply(lambda x: x.tolist())

# Save as CSV
context_df.to_csv("processed_data.csv", index=False)

# Check if it works
print(len(context_df["context_vector"].iloc[0]))  # Should print: 768


768


In [239]:
print(context_df["context_vector"].head())


0    [-0.3729593753814697, 0.02656177058815956, 0.3...
1    [-0.3715858459472656, 0.027815070003271103, 0....
2    [-0.373137891292572, 0.028199542313814163, 0.3...
3    [-0.3731759488582611, 0.026155758649110794, 0....
4    [-0.37287023663520813, 0.026714801788330078, 0...
Name: context_vector, dtype: object


In [241]:
invalid_rows = context_df[~context_df["context_vector"].apply(lambda x: isinstance(x, list) and len(x) == 768)]
print(invalid_rows)



Empty DataFrame
Columns: [Input, Target, context_vector]
Index: []


In [242]:
# Look at the first few problematic rows (you can adjust the index)
print(invalid_rows["context_vector"].head())


Series([], Name: context_vector, dtype: object)


In [243]:
# Fix None or malformed rows by setting them to an empty list (or some other default value)
context_df["context_vector"] = context_df["context_vector"].apply(
    lambda x: [] if not isinstance(x, list) or len(x) != 768 else x
)


In [245]:
loaded_df = pd.read_csv("processed_data.csv")
print(loaded_df.head())

                                               Input  \
0  Moira: "John, I've been stripped of every mors...   
1  Alexis: "Baby, it's crazy, people are just lik...   
2   Revenue agent: "Please sir, can you step aside?"   
3  Moira: "My very soul has been kidnapped, there...   
4  Moira: "My very soul has been kidnapped, there...   

                                              Target  \
0  Johnny: "Well, how do you think I feel, Moira?...   
1   Revenue agent: "Please ma'am, let us do our job"   
2  David: "No, you step aside! You step aside! I'...   
3  Johnny: "We've got 15 minutes to collect our p...   
4  Moira: "Did you put Kristen with Robin? They d...   

                                      context_vector  
0  [-0.3729593753814697, 0.02656177058815956, 0.3...  
1  [-0.3715858459472656, 0.027815070003271103, 0....  
2  [-0.373137891292572, 0.028199542313814163, 0.3...  
3  [-0.3731759488582611, 0.026155758649110794, 0....  
4  [-0.37287023663520813, 0.026714801788330078, 0..

In [248]:
import torch

# Check if any of the tensors in 'context_vector' column contain NaN or infinity
tensor_check = context_df["context_vector"].apply(lambda x: torch.isnan(torch.tensor(x)).any() or torch.isinf(torch.tensor(x)).any())

# Convert the Series of boolean tensors to a regular boolean Series
tensor_check = tensor_check.apply(lambda x: x.item() if isinstance(x, torch.Tensor) else x)

# Print rows with invalid tensors
invalid_tensor_rows = context_df[tensor_check]

# If there are any, print them
print(invalid_tensor_rows)



Empty DataFrame
Columns: [Input, Target, context_vector]
Index: []


In [229]:
import ast
import ast

def is_valid_literal(x):
    try:
        ast.literal_eval(x)
        return True
    except (ValueError, SyntaxError):
        return False

# Now, find the problematic rows
problematic_rows = context_df[context_df['context_vector'].apply(lambda x: not is_valid_literal(x))]
print(problematic_rows.head())



                                               Input  \
0  Moira: "John, I've been stripped of every mors...   
1  Alexis: "Baby, it's crazy, people are just lik...   
2   Revenue agent: "Please sir, can you step aside?"   
3  Moira: "My very soul has been kidnapped, there...   
4  Moira: "My very soul has been kidnapped, there...   

                                              Target context_vector  
0  Johnny: "Well, how do you think I feel, Moira?...           None  
1   Revenue agent: "Please ma'am, let us do our job"           None  
2  David: "No, you step aside! You step aside! I'...           None  
3  Johnny: "We've got 15 minutes to collect our p...           None  
4  Moira: "Did you put Kristen with Robin? They d...           None  


In [214]:
import ast
try:
    context_df["context_vector"] = context_df["context_vector"].apply(lambda x: torch.tensor(ast.literal_eval(x), dtype=torch.float32))
except Exception as e:
    print(f"Error processing context vector: {e}")


Error processing context vector: malformed node or string: [-0.3729593753814697, 0.02656177058815956, 0.3711779713630676, 0.1296720951795578, 0.20723120868206024, 0.26257404685020447, -0.3127001225948334, -0.10862856358289719, -0.13089753687381744, -0.2148495316505432, -0.006360974162817001, 0.37221968173980713, 0.18582910299301147, 0.07218620926141739, 0.40863049030303955, 0.2565861642360687, -0.08756811916828156, 0.06037868559360504, 0.1976148933172226, -0.5320667028427124, -0.13431324064731598, 0.030490543693304062, 0.382279634475708, -0.30864113569259644, 0.17375585436820984, 0.3349620997905731, 0.09747456759214401, -0.250383198261261, -0.3094995617866516, 0.350467324256897, -0.08045945316553116, -0.09895725548267365, 0.27357718348503113, -0.3834195137023926, -0.05679403990507126, -0.19239583611488342, -0.14055898785591125, 0.13765472173690796, 0.46813973784446716, -0.21843644976615906, 0.6420823931694031, 0.01354575902223587, 0.1683768332004547, -0.16114801168441772, -0.3359343111

In [249]:
context_df.head()

,Input,Target,context_vector
0,"Moira: ""John, I've been stripped of every mors...","Johnny: ""Well, how do you think I feel, Moira?...","[-0.3729593753814697, 0.02656177058815956, 0.3..."
1,"Alexis: ""Baby, it's crazy, people are just lik...","Revenue agent: ""Please ma'am, let us do our job""","[-0.3715858459472656, 0.027815070003271103, 0...."
2,"Revenue agent: ""Please sir, can you step aside?""","David: ""No, you step aside! You step aside! I'...","[-0.373137891292572, 0.028199542313814163, 0.3..."
3,"Moira: ""My very soul has been kidnapped, there...","Johnny: ""We've got 15 minutes to collect our p...","[-0.3731759488582611, 0.026155758649110794, 0...."
4,"Moira: ""My very soul has been kidnapped, there...","Moira: ""Did you put Kristen with Robin? They d...","[-0.37287023663520813, 0.026714801788330078, 0..."


In [252]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
df = pd.read_csv("processed_data.csv")
print(df.info)

# Check the initial number of rows
print(f"Total rows in processed_data.csv: {len(df)}")

# Check for any missing or invalid rows
print("Checking for missing or invalid rows:")
print(df.isnull().sum())  # This will show the count of missing values for each column

# Check if there are any duplicates in the dataset
print("Checking for duplicates:")
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Split the data into training and validation sets (80-20 split as an example)
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# Print the sizes of the splits
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")

# Check if there is any leftover data or misalignment
leftover_data = len(df) - (len(train_data) + len(val_data))
print(f"Leftover rows after split: {leftover_data}")

# Check if the split looks correct
print("Rows in training set:")
print(train_data.head())
print("\nRows in validation set:")
print(val_data.head())

# Optional: Check specific data points like the first few and last few entries in the original dataset
print("\nFirst few rows in the original dataset:")
print(df.head())
print("\nLast few rows in the original dataset:")
print(df.tail())


<bound method DataFrame.info of                                                   Input  \
0     Moira: "John, I've been stripped of every mors...   
1     Alexis: "Baby, it's crazy, people are just lik...   
2      Revenue agent: "Please sir, can you step aside?"   
3     Moira: "My very soul has been kidnapped, there...   
4     Moira: "My very soul has been kidnapped, there...   
...                                                 ...   
6145  Ivan: "Sorry I'm late, but I have morning muff...   
6146  Twyla: "Wow, this could be very exciting. You ...   
6147  Roland: "So, there are carols playing, and it ...   
6148  Moira: "I'm inclined to side with Veronica on ...   
6149  Ivan: "This is where I bake. I have three oven...   

                                                 Target  \
0     Johnny: "Well, how do you think I feel, Moira?...   
1      Revenue agent: "Please ma'am, let us do our job"   
2     David: "No, you step aside! You step aside! I'...   
3     Johnny: "We've go

In [250]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import ast

# Load CSV
csv_file = "processed_data.csv"
df = pd.read_csv(csv_file)

# Check if the 'context_vector' column exists
if "context_vector" not in df.columns:
    print("Error: 'context_vector' column not found in the DataFrame.")
else:
    print(f"'context_vector' column found with {df['context_vector'].isnull().sum()} NaN values.")

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token

# Convert context vectors from string to tensors
def safe_eval(x):
    try:
        return torch.tensor(ast.literal_eval(x), dtype=torch.float32)
    except Exception as e:
        print(f"Error processing context vector: {e}")
        return None

df["context_vector"] = df["context_vector"].apply(safe_eval)

# Check for NaN values in context vectors
invalid_contexts = df[df["context_vector"].isnull()]
if not invalid_contexts.empty:
    print(f"Warning: {len(invalid_contexts)} rows have invalid context vectors (NaN or parsing errors).")
    print(invalid_contexts)
else:
    print("All context vectors are valid.")

# Check if there are any other NaN values in the DataFrame
nan_rows = df[df.isnull().any(axis=1)]
if not nan_rows.empty:
    print(f"Warning: {len(nan_rows)} rows contain NaN values in other columns.")
else:
    print("No NaN values found in other columns.")

class ContextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=64):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        context_vector = row["context_vector"]  # 768-dim tensor
        input_text = row["Input"]
        target_text = row["Target"]

        # Tokenize input text
        input_encoding = self.tokenizer(
            input_text, max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt"
        )

        # Tokenize target response (to train GPT-2)
        target_encoding = self.tokenizer(
            target_text, max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt"
        )

        return {
            "context_vector": context_vector,  # (768,)
            "input_ids": input_encoding["input_ids"].squeeze(0),
            "attention_mask": input_encoding["attention_mask"].squeeze(0),
            "target_ids": target_encoding["input_ids"].squeeze(0),
            "target_attention_mask": target_encoding["attention_mask"].squeeze(0),
        }

# Create dataset and dataloaders
dataset = ContextDataset(df, tokenizer)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Print dataset size and a sample of the dataset to check
print(f"Dataset contains {len(df)} rows.")
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"First sample in training dataset: {train_dataset[0]}")


'context_vector' column found with 0 NaN values.
All context vectors are valid.
No NaN values found in other columns.
Dataset contains 6150 rows.
Training set size: 4920
Validation set size: 1230
First sample in training dataset: {'context_vector': tensor([-3.7300e-01,  2.6454e-02,  3.7142e-01,  1.2918e-01,  2.0673e-01,
         2.6194e-01, -3.1281e-01, -1.0816e-01, -1.3153e-01, -2.1485e-01,
        -5.3725e-03,  3.7098e-01,  1.8594e-01,  7.2033e-02,  4.0838e-01,
         2.5607e-01, -8.8446e-02,  5.9887e-02,  1.9707e-01, -5.3259e-01,
        -1.3457e-01,  3.1328e-02,  3.8282e-01, -3.0874e-01,  1.7460e-01,
         3.3454e-01,  9.8504e-02, -2.4950e-01, -3.0765e-01,  3.5202e-01,
        -8.0689e-02, -9.8075e-02,  2.7321e-01, -3.8457e-01, -5.6400e-02,
        -1.9332e-01, -1.3999e-01,  1.3715e-01,  4.6835e-01, -2.1816e-01,
         6.4214e-01,  1.3515e-02,  1.6783e-01, -1.6107e-01, -3.3508e-01,
         1.5315e-01, -5.5131e-01, -5.2216e-03, -2.1638e-01, -4.7997e-01,
        -3.7016e-01, 

In [210]:
from transformers import GPT2LMHeadModel

# Re-download model after clearing cache
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [253]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Modify the input embedding layer to accept context vectors
class GPT2WithContext(nn.Module):
    def __init__(self, gpt2_model, context_dim=768):
        super(GPT2WithContext, self).__init__()
        self.gpt2 = gpt2_model
        self.context_projection = nn.Linear(context_dim, self.gpt2.config.n_embd)  # Project context to GPT-2 hidden size

    def forward(self, input_ids, attention_mask, context_vector):
        # Print the input sizes for debugging
        print(f"Input IDs shape: {input_ids.shape}")
        print(f"Attention mask shape: {attention_mask.shape}")
        print(f"Context vector shape: {context_vector.shape}")
        
        # Project context vector to match GPT-2 hidden size
        projected_context = self.context_projection(context_vector)  # Shape: (batch_size, hidden_size)
        
        # Print the shape of the projected context
        print(f"Projected context shape: {projected_context.shape}")
        
        # Add context to token embeddings
        token_embeddings = self.gpt2.transformer.wte(input_ids)  # GPT-2 token embeddings
        combined_embeddings = token_embeddings + projected_context.unsqueeze(1)  # Broadcast across sequence length
        
        # Print the shape of the combined embeddings
        print(f"Combined embeddings shape: {combined_embeddings.shape}")
        
        # Forward pass through GPT-2
        outputs = self.gpt2(inputs_embeds=combined_embeddings, attention_mask=attention_mask)
        
        # Return outputs
        return outputs

# Initialize modified GPT-2 model with context integration
model = GPT2WithContext(model)

# Check for available device (MPS or CPU)
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")
model.to(device)


Using device: mps


GPT2WithContext(
  (gpt2): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2SdpaAttention(
            (c_attn): Conv1D(nf=2304, nx=768)
            (c_proj): Conv1D(nf=768, nx=768)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D(nf=3072, nx=768)
            (c_proj): Conv1D(nf=768, nx=3072)
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_fe

In [ ]:

import torch.optim as optim
import torch

# Define optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Training loop
epochs = 3
for epoch in range(epochs):
    print(f"\nStarting epoch {epoch + 1}/{epochs}...")
    model.train()
    total_train_loss = 0

    for batch_idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()

        # Move to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        target_ids = batch["target_ids"].to(device)
        context_vector = batch["context_vector"].to(device)

        # Debug: Check for NaNs in inputs
        if torch.isnan(input_ids).any():
            print(f"NaN detected in input_ids at batch {batch_idx}")
        if torch.isnan(attention_mask).any():
            print(f"NaN detected in attention_mask at batch {batch_idx}")
        if torch.isnan(target_ids).any():
            print(f"NaN detected in target_ids at batch {batch_idx}")
        if torch.isnan(context_vector).any():
            print(f"NaN detected in context_vector at batch {batch_idx}")

        # Forward pass
        try:
            outputs = model(input_ids, attention_mask, context_vector)
            logits = outputs.logits
        except Exception as e:
            print(f"Error during forward pass at batch {batch_idx}: {e}")
            continue

        # Compute loss
        try:
            loss = criterion(logits.view(-1, logits.size(-1)), target_ids.view(-1))
        except Exception as e:
            print(f"Error during loss calculation at batch {batch_idx}: {e}")
            continue

        total_train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()  # Added to ensure no gradients accumulate
        loss.backward()

        # Check for NaNs in gradients
        if torch.isnan(loss).any():
            print(f"NaN detected in loss at batch {batch_idx}")
        if any(torch.isnan(param.grad).any() for param in model.parameters()):
            print(f"NaN detected in gradients at batch {batch_idx}")

        optimizer.step()

        # Debug: Print every 100 batches
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx} - Loss: {loss.item():.6f}")

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Training Loss: {avg_train_loss:.6f}")

    # Validation phase
    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            target_ids = batch["target_ids"].to(device)
            context_vector = batch["context_vector"].to(device)

            # Debug: Check for NaNs in validation inputs
            if torch.isnan(input_ids).any():
                print(f"NaN detected in input_ids at validation batch {batch_idx}")
            if torch.isnan(attention_mask).any():
                print(f"NaN detected in attention_mask at validation batch {batch_idx}")
            if torch.isnan(target_ids).any():
                print(f"NaN detected in target_ids at validation batch {batch_idx}")
            if torch.isnan(context_vector).any():
                print(f"NaN detected in context_vector at validation batch {batch_idx}")

            # Forward pass
            try:
                outputs = model(input_ids, attention_mask, context_vector)
                logits = outputs.logits
            except Exception as e:
                print(f"Error during forward pass at validation batch {batch_idx}: {e}")
                continue

            # Compute loss
            try:
                loss = criterion(logits.view(-1, logits.size(-1)), target_ids.view(-1))
            except Exception as e:
                print(f"Error during loss calculation at validation batch {batch_idx}: {e}")
                continue

            total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f"Epoch {epoch + 1} - Validation Loss: {avg_val_loss:.6f}")

# Save fine-tuned GPT-2
# After training, save the entire model using save_pretrained
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")  # Optionally save the tokenizer as well
print("GPT-2 model and tokenizer saved successfully!")
torch.save(model.state_dict(), "fine_tuned_gpt2.pth")
print("GPT-2 Decoder fine-tuned and saved successfully!")




Starting epoch 1/3...
Epoch 1 - Training Loss: nan
Epoch 1 - Validation Loss: nan

Starting epoch 2/3...
Epoch 2 - Training Loss: nan
Epoch 2 - Validation Loss: nan

Starting epoch 3/3...


KeyboardInterrupt: 